## load 参数

In [1]:
import numpy as np
import cv2
import scipy.io as scio

Params=scio.loadmat('workspace.mat')
K = np.array(Params['K'])
K = K.T
distortion=np.array(Params['distortion'])

## sift特征匹配

In [2]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

imgpath = 'images/'
MIN_MATCH_COUNT = 10

def match_pts(imgname1, imgname2):
    img1 = cv2.imread(imgpath+imgname1, 0)          # queryImage
    img2 = cv2.imread(imgpath+imgname2, 0)        # trainImage

    # Initiate SIFT detector
    sift = cv2.xfeatures2d.SIFT_create()

    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)

    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)

    flann = cv2.FlannBasedMatcher(index_params, search_params)

    matches = flann.knnMatch(des1,des2,k=2)

    # store all the good matches as per Lowe's ratio test.
    good = []
    pts1 = []
    pts2 = []
    
    # ratio test as per Lowe's paper
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.8*n.distance:
            good.append(m)
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)

    pts1 = np.int32(pts1)
    pts2 = np.int32(pts2)
    # return the np.array type
    return pts1, pts2

## E F 矩阵 

In [17]:
from numpy import *
from scipy import *
from scipy.linalg import *
from pylab import *
import cv2

def find_fundamental(pts1, pts2):
    F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.FM_LMEDS)
    return F
    
def find_essential(F, K1, K2):
    return K2.T @ F @ K1

def skew(a):
    #Skew matrix A such that cross(a,v) = Av for any v
    return array([[0,-a[2],a[1]],[a[2],0,-a[0]],[-a[1],a[0],0]])

def find_rotation(E, pts1, pts2):
    # make sure E is rank 2
    U,S,V = svd(E)
    if det(dot(U,V)) < 0:
        V = -V
    E = dot(U,dot(diag([1,1,0]),V))    
    
    Z = skew([0,0,-1])
    W = array([[0,-1,0],[1,0,0],[0,0,1]])
    
    # return all four solutions
    R = [vstack((dot(U,dot(W,V)).T,U[:,2])).T,
            vstack((dot(U,dot(W,V)).T,-U[:,2])).T,
            vstack((dot(U,dot(W.T,V)).T,U[:,2])).T,
            vstack((dot(U,dot(W.T,V)).T,-U[:,2])).T]
    
    
    return R

def find_pose():
    


## 运行

In [20]:
pts1, pts2 = match_pts('shoe_orange1.jpg', 'shoe_orange2.jpg')
F = find_fundamental(pts1, pts2)
E = find_essential(F, K, K)
print(F)
print(K)
print(E)

P = find_pose(E)
print(P[0])

[[ 3.09258042e-06 -7.62540326e-05 -1.44913773e-02]
 [ 7.94349267e-05  1.56148691e-06 -2.28442415e-02]
 [ 1.10188455e-02  2.36061682e-02  1.00000000e+00]]
[[966.23930753   0.         583.67813597]
 [  0.         964.93489257 723.88506865]
 [  0.           0.           1.        ]]
[[  2.88728998 -71.09605899 -65.59360112]
 [ 74.06179112   1.45389944  23.78615216]
 [ 67.95143095 -19.07802338   2.74049969]]
[[ 0.99926365  0.02187076  0.031525   -0.20791894]
 [-0.02210174  0.99973124  0.00699687 -0.65686901]
 [-0.0313635  -0.00768848  0.99947847  0.72477087]]
